Listings aus iX Artikel
============

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf

#graphen erzeugen

node_const_1 = tf.constant(3.0, tf.float32, name = 'const_1')
node_const_2 = tf.constant(4.0, tf.float32, name = 'const_2')

node_add = tf.add(node_const_1, node_const_2)

a = tf.placeholder(tf.float32, name = 'a')
node_mul = node_add * a

b = tf.Variable(1.0, tf.float32, name = 'b')
node_sub = node_mul - b


#ausführen

with tf.Session() as session:
    writer = tf.summary.FileWriter('./first_step', session.graph)
    init = tf.global_variables_initializer()
    session.run(init)
    print(session.run(node_sub, {a:2.0}))
    
writer.close()

idx 2 numpy script
==========

Einlesen:
<https://gist.github.com/tylerneylon/ce60e8a06e7506ac45788443f7269e40>

Ausgeben:
<https://python-programmieren.com/deep-learning-mnist-database/>


In [ ]:
!pip install numpy
!pip install matplotlib

In [ ]:
import struct
import numpy as np
from matplotlib import pyplot as plt
import gzip

bilder_idx = "train-images-idx3-ubyte.gz"
bezeichnungen_idx = "train-labels-idx1-ubyte.gz"
bild_nr = 3

with gzip.open(bilder_idx) as f:
    zero, data_type, dims = struct.unpack('>HBB', f.read(4))
    shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
    bilder = np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)

with gzip.open(bezeichnungen_idx) as f:
    zero, data_type, dims = struct.unpack('>HBB', f.read(4))
    shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
    bezeichnungen = np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)
    
 
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(bilder[bild_nr], cmap='gray')
ax.set_title("".join(["Number: ", str(bezeichnungen[bild_nr])]))